## Background

Uncertainty is a facininating concept. It is extremely vast and applies to almost all aspects of life. You could spend your entire life learning about it and still not understand all of it.
Uncertainty is everywhere, in everything we do. For example, uncertainty affects the economy in many ways. Peoples attitude to spending are usually based on their level of confidence in the economy. Uncertainty of our future climate drives us to recycle more, burn more efficient fuel, reduce our emmisions and respect the environment. In a sense, it is almost motivational as once we are aware of something, we typically strive to improve it.
In science, we frequently measure or observe things. We do this so that we can understand them better. By putting a measure on something it allows us to describe something, to visualise it or to comprehend it. Humans will design an experiment or build a model to test or measure a phenomenon and uncertainty will be the measure of the doubt that exists about the result or prediction. 

## Project - Dataset

This project will focus on the field of dimensional metrology and more specifically, measurement systems. A measurement system might be described as a system encompassing the measuring instrument, the software, firmware, people/users, environment, method and the articles that they inspect. When designing a measurement system many aspects must be considered but ultimately the system should have an acceptable accuracy and variance for what it is designed to inspect. In conjunction with this, each measurement result should be accompanied by an uncertainty value since no measurement is ever perfect. When a measurement is taken, many phenomena can occur to influence that result and generally speaking that influence could be captured in the overall uncertainty. Estimating this uncertainy, it is possible to mitigate for the risk of misclassification based off results/observations made.         

## Scenario

You are asked to provide consultation to an aerospace company that manufactures a component. Post manufacture it inspects each component dimensionally to ensure it meets design intent/criteria. The component itself has 4 geometric features that must be verified through inspection, each having a nominal size and an associated tolerance. Two measurement systems are used to inspect the components to meet volume requirements. Both systems are returning vastly different observations. It is not known which instrument is correct. What is going on?

The brief of the project is to create a synthetic dataset. This dataset will attempt to capture what the above situation might look like (in a fairly simplistic approach with assumptions) if it were a real investigation and try to answer some additional questions along the way.

## Investigation

Firstly, the assumption that one of the instruments is correct (in the brief given above) is not advisable. 
Both systems might have issues.

Investigation starts with the component. It has four geometric features that are inspected. They are relatively simple in terms of geometric complexity. Two features are lengths, one is a width and other is a diameter. There are no complex surfaces or advanced geometric constraints to be inspected.

The features are:

length_1 is 14.010mm +/- 0.020
length_2 is 10.050mm +/- 0.050
width_1 is  50.200mm +/- 0.100
Dia_1 is 30mm +/- 0.070

Instruments:

It was found that they are both coordinate measuring machines, often referred to as 'CMM's'. For now lets call them instrument_1, instrument_2
Each has a calibration certificate stating the equipments maximum permissable error for error of length indication. After speaking with the OEM, it is understood that this is essentially an accuracy statement for the equipment when measuring length type features.

It is stated in linear type form:  MPEE  = X + L/K   where X and K are constants. (x = um, L = mm, K = no units)

instrument_1 = 2.4 + L/1000
instrument_2 = 5.0 + L/1000


The certificates show that instrument_1 was calibrated and serviced 2 months ago and instrument_2 was calibrated 9 months ago. It is noted that instrument_2 has a bias of 0.5um. Neither instruments have a linearity issue.


Users:

Looking at users of the measurement systems, It was discovered that each system was run by a different operator.

Mary ran instrument_1 and was quite experienced.
Michael ran instrument_2 and was new in the job. He was quite inexperienced.

One other notable finding was that Michael was left handed while Mary was right handed. This had not been considered when designing the system.


Environment:

instrument_1 was located in a temperature controlled room cerified at 19-21 deg C.
instrument_2 was located in a temperature controlled room cerified at 19-21 deg C however it was found that the climate control unit was broken and had not being repaired. The actual temperature range was 16-24 deg C.